In [2]:
import warnings
warnings.filterwarnings("ignore")

In [19]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns #visualisation
import matplotlib.pyplot as plt #visualisation
%matplotlib inline
sns.set(color_codes=True)

import statistics as stats

from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
# Load the dataset without a header and assign column names
df_tweets = pd.read_csv('processed_tweets_with_sentiment.csv')

In [5]:
# Check that the dataset was uploaded correctly
df_tweets.head(5)

,Unnamed: 0,date,text,processed_text,raw_sentiment,raw_sentiment_classification,processed_sentiment,processed_sentiment_classification,sentiment_difference
0,0,2009-04-06 22:19:45,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http awww bummer shoulda got david ...,-0.0173,Negative,-0.3818,Negative,0.3645
1,1,2009-04-06 22:19:49,is upset that he can't update his Facebook by ...,upset ca update facebook texting might cry res...,-0.7500,Negative,-0.7269,Negative,-0.0231
2,2,2009-04-06 22:19:53,@Kenichan I dived many times for the ball. Man...,kenichan dived many time ball managed save 50 ...,0.4939,Negative,0.4939,Negative,0.0000
3,3,2009-04-06 22:19:57,my whole body feels itchy and like its on fire,whole body feel itchy like fire,-0.2500,Negative,-0.2500,Negative,0.0000
4,4,2009-04-06 22:19:57,"@nationwideclass no, it's not behaving at all....",nationwideclass behaving mad ca see,-0.4939,Negative,-0.4939,Negative,0.0000


In [6]:
# Checking the types of data
df_tweets.dtypes

Unnamed: 0                              int64
date                                   object
text                                   object
processed_text                         object
raw_sentiment                         float64
raw_sentiment_classification           object
processed_sentiment                   float64
processed_sentiment_classification     object
sentiment_difference                  float64
dtype: object

In [7]:
# Preprocess data
# Convert 'date' column to datetime and set as index
df_tweets['date'] = pd.to_datetime(df_tweets['date'])
df_tweets.set_index('date', inplace=True)
series = df_tweets['raw_sentiment'] 

In [18]:
# Checking the types of data
df_tweets.dtypes

Unnamed: 0                              int64
text                                   object
processed_text                         object
raw_sentiment                         float64
raw_sentiment_classification           object
processed_sentiment                   float64
processed_sentiment_classification     object
sentiment_difference                  float64
raw_sentiment_negative                  int64
raw_sentiment_positive                  int64
processed_sentiment_negative            int64
processed_sentiment_positive            int64
dtype: object

In [8]:
# Function to evaluate model
def evaluate_forecast(actual, predicted):
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    return rmse

## Arima Model

In [10]:
# ARIMA Model
arima_model = ARIMA(series, order=(5,1,0))
arima_result = arima_model.fit()

arima_forecast_1 = arima_result.forecast(steps=1)
arima_rmse_1 = evaluate_forecast(series[-1:], arima_forecast_1)

arima_forecast_3 = arima_result.forecast(steps=3)
arima_rmse_3 = evaluate_forecast(series[-3:], arima_forecast_3)

arima_forecast_7 = arima_result.forecast(steps=7)
arima_rmse_7 = evaluate_forecast(series[-7:], arima_forecast_7)

arima_forecast_28 = arima_result.forecast(steps=28)
arima_rmse_28 = evaluate_forecast(series[-28:], arima_forecast_28)

C:\Users\Clara\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Clara\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Clara\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Clara\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Clara\anaconda3\Lib\site-

In [13]:
arima_forecast_1

1600000    0.563919
dtype: float64

In [14]:
arima_forecast_3

1600000    0.563919
1600001    0.525923
1600002    0.522974
Name: predicted_mean, dtype: float64

In [15]:
arima_forecast_7

1600000    0.563919
1600001    0.525923
1600002    0.522974
1600003    0.537465
1600004    0.566669
1600005    0.548205
1600006    0.544218
Name: predicted_mean, dtype: float64

In [17]:
arima_forecast_28

1600000    0.563919
1600001    0.525923
1600002    0.522974
1600003    0.537465
1600004    0.566669
1600005    0.548205
1600006    0.544218
1600007    0.540897
1600008    0.543384
1600009    0.546813
1600010    0.548359
1600011    0.545322
1600012    0.544833
1600013    0.544931
1600014    0.545605
1600015    0.545978
1600016    0.545838
1600017    0.545419
1600018    0.545434
1600019    0.545534
1600020    0.545635
1600021    0.545640
1600022    0.545583
1600023    0.545541
1600024    0.545561
1600025    0.545582
1600026    0.545590
1600027    0.545583
Name: predicted_mean, dtype: float64

## LTSM Model

### Preprocess data

In [11]:
# Define the conditions for the new columns
df_tweets['raw_sentiment_negative'] = df_tweets['raw_sentiment_classification'].apply(lambda x: 1 if x == 'Negative' else 0)
df_tweets['raw_sentiment_positive'] = df_tweets['raw_sentiment_classification'].apply(lambda x: 1 if x == 'Positive' else 0)

df_tweets['processed_sentiment_negative'] = df_tweets['processed_sentiment_classification'].apply(lambda x: 1 if x == 'Negative' else 0)
df_tweets['processed_sentiment_positive'] = df_tweets['processed_sentiment_classification'].apply(lambda x: 1 if x == 'Positive' else 0)

In [12]:
# Check that the dataset was updated correctly
df_tweets.head(5)

,Unnamed: 0,text,processed_text,raw_sentiment,raw_sentiment_classification,processed_sentiment,processed_sentiment_classification,sentiment_difference,raw_sentiment_negative,raw_sentiment_positive,processed_sentiment_negative,processed_sentiment_positive
date,,,,,,,,,,,,
2009-04-06 22:19:45,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http awww bummer shoulda got david ...,-0.0173,Negative,-0.3818,Negative,0.3645,1,0,1,0
2009-04-06 22:19:49,1,is upset that he can't update his Facebook by ...,upset ca update facebook texting might cry res...,-0.7500,Negative,-0.7269,Negative,-0.0231,1,0,1,0
2009-04-06 22:19:53,2,@Kenichan I dived many times for the ball. Man...,kenichan dived many time ball managed save 50 ...,0.4939,Negative,0.4939,Negative,0.0000,1,0,1,0
2009-04-06 22:19:57,3,my whole body feels itchy and like its on fire,whole body feel itchy like fire,-0.2500,Negative,-0.2500,Negative,0.0000,1,0,1,0
2009-04-06 22:19:57,4,"@nationwideclass no, it's not behaving at all....",nationwideclass behaving mad ca see,-0.4939,Negative,-0.4939,Negative,0.0000,1,0,1,0


We need numerical sentiment classification in order to be able to run the model

In [21]:
# Ensure all entries in 'processed_text' are strings and handle missing values
df_tweets['processed_text'] = df_tweets['processed_text'].astype(str)

In [29]:
# Ensure 'processed_sentiment' is the target variable for time-series forecasting
series = df_tweets['processed_sentiment']
texts = df_tweets['processed_text'].astype(str).values

### Text Vectorization

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Vectorize the text data with reduced features
vectorizer = TfidfVectorizer(max_features=1000)  # Reduce max_features to manage memory
text_vectors = vectorizer.fit_transform(texts)  # Keep it as a sparse matrix

print("Shape of text_vectors:", text_vectors.shape)

Shape of text_vectors: (1600000, 1000)


### Combine Text Vectors with Sentiment Data

In [32]:
# Scale the sentiment data
scaler = MinMaxScaler()
series_scaled = scaler.fit_transform(series.values.reshape(-1, 1))

In [33]:
# Function to create sequences with text vectors and sentiment values
def create_sequences_with_text(data, text_vectors, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        sentiment_seq = data[i:i + seq_length]
        text_vector = text_vectors[i + seq_length].toarray()
        combined_seq = np.hstack((sentiment_seq.flatten(), text_vector.flatten()))
        X.append(combined_seq)
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

In [34]:
# Create sequences
seq_length = 5
X, y = create_sequences_with_text(series_scaled, text_vectors, seq_length)

In [35]:
# Split the data into training and test sets
split_ratio = 0.8
split_index = int(len(X) * split_ratio)
X_train, y_train = X[:split_index], y[:split_index]
X_test, y_test = X[split_index:], y[split_index:]

print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_test:", y_test.shape)

Shape of X_train: (1279996, 1005)
Shape of y_train: (1279996, 1)
Shape of X_test: (319999, 1005)
Shape of y_test: (319999, 1)


### Build and Train the LSTM Model

In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten

# Build the LSTM model
model = Sequential()
model.add(Dense(50, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test), verbose=1)



Epoch 1/50

20000/20000 [==============================] - 37s 2ms/step - loss: 0.0128 - val_loss: 0.0087
Epoch 2/50
20000/20000 [==============================] - 30s 2ms/step - loss: 0.0108 - val_loss: 0.0084
Epoch 3/50
20000/20000 [==============================] - 31s 2ms/step - loss: 0.0107 - val_loss: 0.0082
Epoch 4/50
20000/20000 [==============================] - 31s 2ms/step - loss: 0.0106 - val_loss: 0.0082
Epoch 5/50
20000/20000 [==============================] - 30s 2ms/step - loss: 0.0105 - val_loss: 0.0081
Epoch 6/50
20000/20000 [==============================] - 30s 2ms/step - loss: 0.0105 - val_loss: 0.0080
Epoch 7/50
20000/20000 [==============================] - 30s 2ms/step - loss: 0.0105 - val_loss: 0.0080
Epoch 8/50
20000/20000 [==============================] - 30s 2ms/step - loss: 0.0105 - val_loss: 0.0080
Epoch 9/50
20000/20000 [==============================] - 30s 2ms/step - loss: 0.0105 - val_loss: 0.0080
Epoch 10/50
20000/20000 [===========================

### Evaluate Model Performance

In [37]:
from sklearn.metrics import mean_squared_error

# Make predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Invert predictions to original scale
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)
y_train_inv = scaler.inverse_transform(y_train.reshape(-1, 1))
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate RMSE
train_rmse = np.sqrt(mean_squared_error(y_train_inv, train_predict))
test_rmse = np.sqrt(mean_squared_error(y_test_inv, test_predict))
print("Train RMSE:", train_rmse)
print("Test RMSE:", test_rmse)

10000/10000 [==============================] - 9s 901us/step
Train RMSE: 0.19407007805740084
Test RMSE: 0.17888239695059494


### Make Forecast for 1 Day, 3 Days, 7 Days and 1 month

In [44]:
# Function to forecast future values with debugging prints
def forecast(model, data, text_vectors, seq_length, steps):
    forecast_data = data[-seq_length:]
    forecasts = []
    for i in range(steps):
        sentiment_seq = forecast_data[-seq_length:]
        
        # Correct index calculation for text_vector
        text_index = len(data) - seq_length + i
        if text_index >= text_vectors.shape[0]:
            text_index = text_vectors.shape[0] - 1
        text_vector = text_vectors[text_index].toarray()
        
        combined_input = np.hstack((sentiment_seq.flatten(), text_vector.flatten()))
        
        # Print shapes for debugging
        print("Step:", i + 1)
        print("Shape of sentiment_seq:", sentiment_seq.shape)
        print("Shape of text_vector:", text_vector.shape)
        print("Shape of combined_input before reshape:", combined_input.shape)
        
        combined_input = combined_input.reshape(1, -1)  # Ensure the input shape is correct
        
        # Print shape after reshaping
        print("Shape of combined_input after reshape:", combined_input.shape)
        
        prediction = model.predict(combined_input)
        forecast_data = np.append(forecast_data[1:], prediction)
        forecasts.append(prediction)
    return np.array(forecasts).reshape(-1, 1)  # Ensure the output is 2D

# Forecast for 1 day, 3 days, 7 days, and 28 days
forecasts_1_day_LSTM = forecast(model, series_scaled, text_vectors, seq_length, 1)
forecasts_3_days_LSTM = forecast(model, series_scaled, text_vectors, seq_length, 3)
forecasts_7_days_LSTM = forecast(model, series_scaled, text_vectors, seq_length, 7)
forecasts_28_days_LSTM = forecast(model, series_scaled, text_vectors, seq_length, 28)

# Invert forecasts to original scale
forecasts_1_day_LSTM = scaler.inverse_transform(forecasts_1_day_LSTM)
forecasts_3_days_LSTM = scaler.inverse_transform(forecasts_3_days_LSTM)
forecasts_7_days_LSTM = scaler.inverse_transform(forecasts_7_days_LSTM)
forecasts_28_days_LSTM = scaler.inverse_transform(forecasts_28_days_LSTM)

print("1-day forecast LSTM:", forecasts_1_day_LSTM.flatten())
print("3-day forecast LSTM:", forecasts_3_days_LSTM.flatten())
print("7-day forecast LSTM:", forecasts_7_days_LSTM.flatten())
print("28-day forecast LSTM:", forecasts_28_days_LSTM.flatten())

Step: 1
Shape of sentiment_seq: (5, 1)
Shape of text_vector: (1, 1000)
Shape of combined_input before reshape: (1005,)
Shape of combined_input after reshape: (1, 1005)
1/1 [==============================] - 0s 17ms/step
Step: 1
Shape of sentiment_seq: (5, 1)
Shape of text_vector: (1, 1000)
Shape of combined_input before reshape: (1005,)
Shape of combined_input after reshape: (1, 1005)
1/1 [==============================] - 0s 17ms/step
Step: 2
Shape of sentiment_seq: (5,)
Shape of text_vector: (1, 1000)
Shape of combined_input before reshape: (1005,)
Shape of combined_input after reshape: (1, 1005)
1/1 [==============================] - 0s 17ms/step
Step: 3
Shape of sentiment_seq: (5,)
Shape of text_vector: (1, 1000)
Shape of combined_input before reshape: (1005,)
Shape of combined_input after reshape: (1, 1005)
1/1 [==============================] - 0s 17ms/step
Step: 1
Shape of sentiment_seq: (5, 1)
Shape of text_vector: (1, 1000)
Shape of combined_input before reshape: (1005,)
Shape

###  Visualize and Report Results

In [45]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

In [70]:
app_arima = dash.Dash(__name__)

app_arima.layout = html.Div([
    html.H1("ARIMA Model Forecasting Dashboard"),
    dcc.Graph(
        id='arima-forecast',
        figure={
            'data': [
                go.Scatter(x=list(range(len(series))), y=series, mode='lines', name='Actual'),
                go.Scatter(x=list(range(len(series), len(series) + 1)), y=arima_forecast_1, mode='lines', name='1-step Forecast'),
                go.Scatter(x=list(range(len(series), len(series) + 3)), y=arima_forecast_3, mode='lines', name='3-step Forecast'),
                go.Scatter(x=list(range(len(series), len(series) + 7)), y=arima_forecast_7, mode='lines', name='7-step Forecast'),
                go.Scatter(x=list(range(len(series), len(series) + 28)), y=arima_forecast_28, mode='lines', name='28-step Forecast')
            ],
            'layout': go.Layout(title='ARIMA Forecasts', xaxis={'title': 'Time'}, yaxis={'title': 'Sentiment'})
        }
    ),
    html.P(f"RMSE (1-step): {arima_rmse_1}"),
    html.P(f"RMSE (3-step): {arima_rmse_3}"),
    html.P(f"RMSE (7-step): {arima_rmse_7}"),
    html.P(f"RMSE (28-step): {arima_rmse_28}")
])

if __name__ == '__main__':
    app_arima.run_server(debug=True, port=8050)

In [51]:
history = {
    'loss': [0.0128, 0.0108, 0.0107, 0.0106, 0.0105, 0.0105, 0.0105, 0.0105, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104, 0.0104],
    'val_loss': [0.0087, 0.0084, 0.0082, 0.0082, 0.0082, 0.0080, 0.0080, 0.0080, 0.0079, 0.0079, 0.0080, 0.0080, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081, 0.0081]
}

In [55]:
# Function to calculate RMSE
def calculate_rmse(actual, predicted):
    return np.sqrt(mean_squared_error(actual, predicted))

# Assuming you have the following data:
# series_scaled - the actual data
# forecasts_1_day_LSTM, forecasts_3_days_LSTM, forecasts_7_days_LSTM, forecasts_28_days_LSTM - predicted data for different horizons

# Extract the relevant portions of the series_scaled data for comparison
actual_1_day = series_scaled[-len(forecasts_1_day_LSTM):]
actual_3_days = series_scaled[-len(forecasts_3_days_LSTM):]
actual_7_days = series_scaled[-len(forecasts_7_days_LSTM):]
actual_28_days = series_scaled[-len(forecasts_28_days_LSTM):]

# Calculate RMSE for each forecast horizon
lstm_rmse_1 = calculate_rmse(actual_1_day, forecasts_1_day_LSTM)
lstm_rmse_3 = calculate_rmse(actual_3_days, forecasts_3_days_LSTM)
lstm_rmse_7 = calculate_rmse(actual_7_days, forecasts_7_days_LSTM)
lstm_rmse_28 = calculate_rmse(actual_28_days, forecasts_28_days_LSTM)

print(f"RMSE (1-day): {lstm_rmse_1}")
print(f"RMSE (3-day): {lstm_rmse_3}")
print(f"RMSE (7-day): {lstm_rmse_7}")
print(f"RMSE (28-day): {lstm_rmse_28}")

RMSE (1-day): 0.13546542629029545
RMSE (3-day): 0.32411547594409457
RMSE (7-day): 0.2724084276585802
RMSE (28-day): 0.1611497658126404


In [71]:
app_lstm = Dash(__name__)

app_lstm.layout = html.Div([
    html.H1("LSTM Model Forecasting Dashboard"),
    dcc.Graph(
        id='lstm-training',
        figure={
            'data': [
                go.Scatter(x=list(range(len(history['loss']))), y=history['loss'], mode='lines', name='Training Loss'),
                go.Scatter(x=list(range(len(history['val_loss']))), y=history['val_loss'], mode='lines', name='Validation Loss')
            ],
            'layout': go.Layout(title='LSTM Training and Validation Loss', xaxis={'title': 'Epochs'}, yaxis={'title': 'Loss'})
        }
    ),
    dcc.Graph(
        id='lstm-forecast',
        figure={
            'data': [
                go.Scatter(x=list(range(len(series_scaled) - len(series_scaled[-1:]), len(series_scaled))), y=series_scaled[-1:].flatten(), mode='lines', name='Actual'),
                go.Scatter(x=list(range(len(series_scaled) - len(series_scaled[-1:]), len(series_scaled) - len(series_scaled[-1:]) + 1)), y=forecasts_1_day_LSTM.flatten(), mode='lines', name='1-day Forecast'),
                go.Scatter(x=list(range(len(series_scaled) - len(series_scaled[-1:]), len(series_scaled) - len(series_scaled[-1:]) + 3)), y=forecasts_3_days_LSTM.flatten(), mode='lines', name='3-day Forecast'),
                go.Scatter(x=list(range(len(series_scaled) - len(series_scaled[-1:]), len(series_scaled) - len(series_scaled[-1:]) + 7)), y=forecasts_7_days_LSTM.flatten(), mode='lines', name='7-day Forecast'),
                go.Scatter(x=list(range(len(series_scaled) - len(series_scaled[-1:]), len(series_scaled) - len(series_scaled[-1:]) + 28)), y=forecasts_28_days_LSTM.flatten(), mode='lines', name='28-day Forecast')
            ],
            'layout': go.Layout(title='LSTM Forecasts', xaxis={'title': 'Time'}, yaxis={'title': 'Sentiment'})
        }
    ),
    html.P(f"RMSE (1-day): {lstm_rmse_1}"),
    html.P(f"RMSE (3-day): {lstm_rmse_3}"),
    html.P(f"RMSE (7-day): {lstm_rmse_7}"),
    html.P(f"RMSE (28-day): {lstm_rmse_28}")
])

if __name__ == '__main__':
    app_lstm.run_server(debug=True, port=8051)

In [64]:
from dash import Dash, html, dcc

# Sampled Data
sample_rate = 1000  # Adjust this number to sample less/more data points
sampled_indices = np.arange(0, len(series), sample_rate)
sampled_series = series[sampled_indices]
sampled_arima_forecast_1 = arima_forecast_1[:len(sampled_indices)]
sampled_arima_forecast_3 = arima_forecast_3[:len(sampled_indices)]
sampled_arima_forecast_7 = arima_forecast_7[:len(sampled_indices)]
sampled_arima_forecast_28 = arima_forecast_28[:len(sampled_indices)]

In [74]:
from dash import Dash, html, dcc
import plotly.graph_objs as go

# Assuming series, sampled_indices, sampled_series, sampled_arima_forecast_1, sampled_arima_forecast_3, sampled_arima_forecast_7, sampled_arima_forecast_28, 
# arima_rmse_1, arima_rmse_3, arima_rmse_7, arima_rmse_28 are already defined

app_arima_1 = Dash(__name__)

app_arima_1.layout = html.Div([
    html.H1("ARIMA Model Forecasting Dashboard (Sampled Data)"),
    dcc.Graph(
        id='arima-forecast-1',
        figure={
            'data': [
                go.Scatter(x=sampled_indices, y=sampled_series, mode='lines', name='Actual'),
                go.Scatter(x=sampled_indices, y=sampled_arima_forecast_1, mode='lines', name='1-step Forecast', line=dict(dash='dash')),
                go.Scatter(x=sampled_indices, y=sampled_arima_forecast_3, mode='lines', name='3-step Forecast', line=dict(dash='dot')),
                go.Scatter(x=sampled_indices, y=sampled_arima_forecast_7, mode='lines', name='7-step Forecast', line=dict(dash='longdash')),
                go.Scatter(x=sampled_indices, y=sampled_arima_forecast_28, mode='lines', name='28-step Forecast', line=dict(dash='dashdot'))
            ],
            'layout': go.Layout(title='ARIMA Forecasts (Sampled Data)', xaxis={'title': 'Time'}, yaxis={'title': 'Sentiment'})
        }
    ),
    html.P(f"RMSE (1-step): {arima_rmse_1}"),
    html.P(f"RMSE (3-step): {arima_rmse_3}"),
    html.P(f"RMSE (7-step): {arima_rmse_7}"),
    html.P(f"RMSE (28-step): {arima_rmse_28}")
])

if __name__ == '__main__':
    app_arima_1.run_server(port=8052, debug=True)

In [66]:
# Zoomed-In Data
zoom_start = -500
zoomed_series = series[zoom_start:]
zoomed_arima_forecast_1 = arima_forecast_1[zoom_start:]
zoomed_arima_forecast_3 = arima_forecast_3[zoom_start:]
zoomed_arima_forecast_7 = arima_forecast_7[zoom_start:]
zoomed_arima_forecast_28 = arima_forecast_28[zoom_start:]

In [73]:
app_arima_2 = Dash(__name__)

app_arima_2.layout = html.Div([
    html.H1("ARIMA Model Forecasting Dashboard (Zoomed-In Data)"),
    dcc.Graph(
        id='arima-forecast-2',
        figure={
            'data': [
                go.Scatter(x=list(range(len(zoomed_series))), y=zoomed_series, mode='lines', name='Actual'),
                go.Scatter(x=list(range(len(zoomed_series), len(zoomed_series) + 1)), y=zoomed_arima_forecast_1, mode='lines', name='1-step Forecast', line=dict(dash='dash')),
                go.Scatter(x=list(range(len(zoomed_series), len(zoomed_series) + 3)), y=zoomed_arima_forecast_3, mode='lines', name='3-step Forecast', line=dict(dash='dot')),
                go.Scatter(x=list(range(len(zoomed_series), len(zoomed_series) + 7)), y=zoomed_arima_forecast_7, mode='lines', name='7-step Forecast', line=dict(dash='longdash')),
                go.Scatter(x=list(range(len(zoomed_series), len(zoomed_series) + 28)), y=zoomed_arima_forecast_28, mode='lines', name='28-step Forecast', line=dict(dash='dashdot'))
            ],
            'layout': go.Layout(title='ARIMA Forecasts (Zoomed-In Data)', xaxis={'title': 'Time'}, yaxis={'title': 'Sentiment'})
        }
    ),
    html.P(f"RMSE (1-step): {arima_rmse_1}"),
    html.P(f"RMSE (3-step): {arima_rmse_3}"),
    html.P(f"RMSE (7-step): {arima_rmse_7}"),
    html.P(f"RMSE (28-step): {arima_rmse_28}")
])

if __name__ == '__main__':
    app_arima_2.run_server(port=8052, debug=True)